In [1]:
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
data = pd.read_csv('trackman.csv')

In [ ]:
data

In [ ]:
data['Batter'].unique()

In [ ]:
batter_dfs = {batter: group for batter, group in data.groupby('Batter')}

In [ ]:
goehrig_df = batter_dfs['Goehrig, Matt']
goehrig_df

In [ ]:
goehrig_df.keys()

In [ ]:
plate_appearances = 0
for num in goehrig_df['PitchofPA']:
    if num == 1.0:
        plate_appearances += 1


In [ ]:
walks = 0
for item in goehrig_df['KorBB']:
    if item == 'Walk':
        walks += 1


In [ ]:
sacrifices = 0
singles = 0
doubles = 0
triples = 0
HRs = 0
hits = 0
for item in goehrig_df['PlayResult']:
    if item == 'Sacrifice':
        scrifices += 1
    if item == 'Single':
        singles += 1
        hits += 1
    if item == 'Double':
        doubles += 1
        hits += 1
    if item == 'Triple':
        triples += 1
        hits += 1
    if item == 'HomeRun':
        HRs += 1
        hits += 1

In [ ]:
batter_stats = []

for batter, batter_df in batter_dfs.items():
   
    plate_appearances = (batter_df['PitchofPA'] == 1.0).sum()
    walks = (batter_df['KorBB'] == 'Walk').sum()
    
    sacrifices = (batter_df['PlayResult'] == 'Sacrifice').sum()
    singles = (batter_df['PlayResult'] == 'Single').sum()
    doubles = (batter_df['PlayResult'] == 'Double').sum()
    triples = (batter_df['PlayResult'] == 'Triple').sum()
    HRs = (batter_df['PlayResult'] == 'HomeRun').sum()
    hits = singles + doubles + triples + HRs
    max_exit_velocity = batter_df['ExitSpeed'].max()
    avg_exit_angle = batter_df['Angle'].mean()
    
    
    batter_stats.append({
        'Batter': batter,
        'PlateAppearances': plate_appearances,
        'Walks': walks,
        'Sacrifices': sacrifices,
        'Singles': singles,
        'Doubles': doubles,
        'Triples': triples,
        'HomeRuns': HRs,
        'Hits': hits,
        'MaxExitVelocity': max_exit_velocity,
        'AverageExitAngle': avg_exit_angle
    })

batter_stats_df = pd.DataFrame(batter_stats)

In [ ]:
def safe_divide(numerator, denominator):
    return numerator / denominator if denominator > 0 else 0

In [ ]:
# Calculate At-Bats (AB)
batter_stats_df['AtBats'] = batter_stats_df['PlateAppearances'] - batter_stats_df['Walks'] - batter_stats_df['Sacrifices']

# Calculate Batting Average (BA)
batter_stats_df['BattingAverage'] = batter_stats_df.apply(
    lambda row: safe_divide(row['Hits'], row['AtBats']), axis=1
)

# Calculate On-Base Percentage (OBP)
batter_stats_df['OnBasePercentage'] = batter_stats_df.apply(
    lambda row: safe_divide(row['Hits'] + row['Walks'], row['PlateAppearances']), axis=1
)


# Calculate Slugging Percentage (SLG)
batter_stats_df['SluggingPercentage'] = batter_stats_df.apply(
    lambda row: safe_divide(row['Singles'] + 2*row['Doubles'] + 3*row['Triples'] + 4*row['HomeRuns'], row['AtBats']), axis=1
)

batter_stats_df['OPS'] = batter_stats_df['OnBasePercentage'] + batter_stats_df['SluggingPercentage']

batter_stats_df['WalkRate'] = batter_stats_df['Walks'] / batter_stats_df['PlateAppearances']

# Display the updated dataframe
batter_stats_df.head()

In [ ]:
marcoux_df = batter_dfs['Marcoux, Noah']


In [ ]:
mask = marcoux_df['PlayResult'] != 'Undefined'
marcoux_df = marcoux_df[mask]

In [ ]:
mask = marcoux_df['PlayResult'] != 'Out'
marcoux_df = marcoux_df[mask]

In [ ]:
marcoux_df.to_csv('data_test1.csv', index=False)

NameError: name 'marcoux_df' is not defined

In [3]:
marcoux_df = marcoux_df[(marcoux_df['Bearing'] >= -45) & (marcoux_df['Bearing'] <= 45)]
marcoux_df['Bearing_Radians'] = np.radians(marcoux_df['Bearing'])

print(marcoux_df['Bearing'].describe())
print(marcoux_df['Bearing'].unique()[:20])  # preview first 20 unique values

# Create figure with polar projection
fig, ax = plt.subplots(figsize=(8, 8), subplot_kw={'projection': 'polar'})
ax.set_theta_zero_location('N')  # Ensure center field is directly upward
ax.set_theta_direction(-1)  # Ensure angles increase clockwise

# Set theta limits for fair territory
ax.set_thetamin(-45)
ax.set_thetamax(45)

# Plot hits
ax.scatter(marcoux_df['Bearing_Radians'], marcoux_df['Distance'], alpha=0.7, c='blue', edgecolors='black')

# Set foul lines at -45° and 45°
foul_lines = np.radians([-45, 45])
max_distance = marcoux_df['Distance'].max() * 1.1  # Extend foul lines beyond data points

# Plot foul lines
for angle in foul_lines:
    ax.plot([angle, angle], [0, max_distance], 'r', linestyle="dashed")

# **Add Baseball Diamond (Home Plate + Bases)**
diamond_distances = [0, 90, 127, 90, 0]  # Home -> 1st -> 2nd -> 3rd -> Home (feet)
diamond_angles = np.radians([0, 45, 90, 135, 180])  # Angles for bases

ax.plot(diamond_angles, diamond_distances, color="black", linewidth=2, label="Base Diamond")

# **Add Outfield Wall (~400 ft)**
wall_distance = 375  # Distance from home plate to outfield wall
wall_angles = np.linspace(np.radians(-45), np.radians(45), 100)  # Smooth curve

ax.plot(wall_angles, [wall_distance] * len(wall_angles), color="darkgreen", linewidth=3, label="Outfield Wall")

# Format the plot to match a baseball field
ax.set_xticks(np.radians([-45, 0, 45]))  # Left field, center field, right field
ax.set_xticklabels(['Left Field', 'Center Field', 'Right Field'])

ax.set_yticklabels([])  # Hide radial labels
ax.set_ylim(0, 420)  # Slightly beyond outfield wall

# Add legend and title
ax.legend()
ax.set_title('Noah Marcoux Spray Chart', fontsize=14)
plt.savefig('spray_chart_draft.png')
plt.show()

NameError: name 'marcoux_df' is not defined

In [ ]:
plt.figure(figsize = (10, 5))
plt.scatter(batter_stats_df['AverageExitAngle'], batter_stats_df['OPS'])
plt.scatter(batter_stats_df['AverageExitAngle'].iloc[25], batter_stats_df['OPS'].iloc[25], color='red')
plt.xlabel('Average Launch Angle')
plt.ylabel('OPS')
plt.title('Correlation Between Launch Angle and OPS')
plt.grid(axis="y", linestyle="-", alpha=0.5)
plt.grid(axis="x", linestyle="-", alpha=0.5)
plt.savefig('LA_OPS_corr_draft.png')

In [ ]:
data = pd.read_csv('trackman.csv')
columns_to_keep = ['Batter','Outs','Balls','Strikes', 'BatterTeam','TaggedPitchType','PlayResult','PlateLocHeight','PlateLocSide']

data_clean = data.loc[:, columns_to_keep]
data_clean = data_clean[data_clean['PlayResult'] != 'Undefined']

In [ ]:
zone_x_edge = (-1, 1)
zone_y_edge = (1.5, 3.5)
x_bins = (zone_x_edge[0], zone_x_edge[1], 4)
y_bins = (zone_y_edge[1], zone_y_edge[1], 4)

In [ ]:
hits = ['Single', 'Double', 'Triple', 'HomeRun']

In [ ]:
batter_mask = data_clean['Batter'] == "Marcoux, Noah"
batter_df = data_clean[batter_mask]
batter_df

In [ ]:
horiz_bin = np.linspace(-1, 1, 4)
vert_bin = np.linspace(1.5, 3.5, 4)
batter_df = batter_df.copy()

In [ ]:
batter_df.loc[:, 'HorizontalBin'] = pd.cut(batter_df['PlateLocSide'], bins=horiz_bin, labels=[0, 1, 2], include_lowest=True)
batter_df.loc[:, 'VerticalBin'] = pd.cut(batter_df['PlateLocHeight'], bins=vert_bin, labels=[0, 1, 2], include_lowest=True)

In [ ]:
bin_groups = batter_df.groupby(['HorizontalBin', 'VerticalBin'])
heatmap_data = bin_groups.apply(lambda x: sum(x['PlayResult'].isin(hits)) / len(x)).unstack()
heatmap_data = heatmap_data.fillna(0)

In [ ]:
plt.figure(figsize=(10, 7))
sns.heatmap(heatmap_data, annot=True, cmap="turbo", linewidths=.5)
plt.title('Noah Marcoux Batting Average Heatmap')
plt.xlabel('Horizontal Location of Pitch')
plt.ylabel('Vertical Location of Pitch')
plt.savefig('heatmap_draft.png')
plt.show()

In [ ]:
batter_stats_df

In [ ]:
selected_batter = "Marcoux, Noah"

# Compute the mean and range for each statistical category
categories = ['BattingAverage','OnBasePercentage', 'MaxExitVelocity', 'WalkRate']
means = batter_stats_df[categories].mean()
mins = batter_stats_df[categories].min()
maxs = batter_stats_df[categories].max()

# Get the selected batter's stats
batter_data = batter_stats_df[batter_stats_df['Batter'] == selected_batter][categories].iloc[0]

# Normalize all stats to be between 0 and 1, centering the average at 0.5
normalized_batter = 0.5 + ((batter_data - means) / (maxs - mins))
normalized_means = 0.5 + ((means - means) / (maxs - mins))  # This ensures means stay at 0.5

# Create the figure
plt.figure(figsize=(8, 5))

# X-axis categories
x = np.arange(len(categories))

# Plot the league average as a horizontal line at 0.5
plt.axhline(0.5, color='gray', linestyle='--', alpha=0.7, label="League Average")

# Plot the player's normalized stats
plt.plot(x, normalized_batter, marker='o', linestyle='-', color='blue', label=selected_batter)

# Label each axis with the league average for that stat
for i, category in enumerate(categories):
    # League average label
    plt.text(x[i], 0.52, f"Avg: {means[category]:.3f}", 
             ha='center', va='bottom', fontsize=10, color='black')

    # Player's value label
    plt.text(x[i], normalized_batter[i] + 0.02, f"{batter_data[category]:.3f}", 
             ha='center', va='bottom', fontsize=10, color='blue', fontweight='bold')

    # Min value label
    plt.text(x[i], 0.02, f"Min: {mins[category]:.3f}", 
             ha='center', va='bottom', fontsize=10, color='red')

    # Max value label
    plt.text(x[i], 0.98, f"Max: {maxs[category]:.3f}", 
             ha='center', va='top', fontsize=10, color='green')

# Labels and aesthetics
plt.xticks(x, categories)  # Set category labels
plt.yticks([0, 0.5, 1], ["Min", "Avg", "Max"])  # Label y-axis
plt.ylabel("Scaled Performance")
plt.title("Parallel Coordinates Plot for Noah Marcoux")
plt.legend(loc = 'lower right')
plt.grid(True, linestyle="--", alpha=0.5)

plt.savefig('parallel_coords_draft.png')
plt.show()